# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-4. rnn_long_sentence

<hr>
``` python 

# Lab 12 RNN
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10  # Any arbitrary number

dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape

# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# (optional) softmax layer
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

'''
0 167 tttttttttt 3.23111
0 168 tttttttttt 3.23111
0 169 tttttttttt 3.23111
…
499 167  of the se 0.229616
499 168 tf the sea 0.229616
499 169   the sea. 0.229616

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

'''



```

<hr>

In [3]:
# Lab 12 RNN
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility


C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [5]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10  # Any arbitrary number

In [8]:
dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)
    

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
batch_size = len(dataX)
batch_size

170

In [12]:
X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

# One-hot encoding, check out the shape
X_one_hot = tf.one_hot(X, num_classes)
X_one_hot  

<tf.Tensor 'one_hot_1:0' shape=(?, 10, 25) dtype=float32>

In [13]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# (optional) softmax layer
X_for_softmax = tf.reshape(outputs, [-1, hidden_size])
softmax_w = tf.get_variable("softmax_w", [hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, seq_length, num_classes])
# All weights are 1 (equal weights)
weights = tf.ones([batch_size, seq_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(mean_loss)


In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
time1 = time.time()

for i in tqdm_notebook(range(500)):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)
        
time2 = time.time()  

A Jupyter Widget

0 0 rrreeeeeer 3.2204998
0 1 eeeeeeeeee 3.2204998
0 2 eeeeeeeeee 3.2204998
0 3 eeeeeeeeee 3.2204998
0 4 eeeeereeee 3.2204998
0 5 eeerrreeee 3.2204998
0 6 eerrrreeee 3.2204998
0 7 rrrrrseeee 3.2204998
0 8 rrrreeeeee 3.2204998
0 9 rreeeeeeee 3.2204998
0 10 eeeeeeeeee 3.2204998
0 11 eeeeeeeeee 3.2204998
0 12 eeeeeeeeee 3.2204998
0 13 eeeeeeeeee 3.2204998
0 14 eeeeeeeeee 3.2204998
0 15 rttetereee 3.2204998
0 16 eeeeeeeeee 3.2204998
0 17 rrrerreeee 3.2204998
0 18 eeeeeeeeee 3.2204998
0 19 rrrreeeeee 3.2204998
0 20 erreeeeeee 3.2204998
0 21 rrreeeeeee 3.2204998
0 22 eeeeeeeeee 3.2204998
0 23 eeeeeeeeee 3.2204998
0 24 eeeesseere 3.2204998
0 25 rrrsrrrrrr 3.2204998
0 26 essrrrrrse 3.2204998
0 27 rsreeeeeee 3.2204998
0 28 eeeeeeeeee 3.2204998
0 29 reereeeeee 3.2204998
0 30 eereseeeee 3.2204998
0 31 rrrsreeeee 3.2204998
0 32 rssseettte 3.2204998
0 33 eeeeeeeeee 3.2204998
0 34 eeeeeeeeee 3.2204998
0 35 rretttteee 3.2204998
0 36 eetttteeee 3.2204998
0 37 etttteeeee 3.2204998
0 38 rrreeeeeee 3.2204

1 141 eeeeeeeeee 3.1105313
1 142 eeeeeeeeee 3.1105313
1 143 eeeeeeeeee 3.1105313
1 144 eeeeeeeeee 3.1105313
1 145 eeeeeeeeee 3.1105313
1 146 eeeeeeeeee 3.1105313
1 147 eeeeeeeeee 3.1105313
1 148 eeeeeeeeee 3.1105313
1 149 eeeeeeeeee 3.1105313
1 150 eeeeeeeeee 3.1105313
1 151 eeeeeeeeee 3.1105313
1 152 eeeeeeeeee 3.1105313
1 153 eeeeeeeeee 3.1105313
1 154 eeeeeeeeee 3.1105313
1 155 eeeeeeeeee 3.1105313
1 156 eeeeeeeeee 3.1105313
1 157 eeeeeeeeee 3.1105313
1 158 eeeeeeeeee 3.1105313
1 159 eeeeeeeeee 3.1105313
1 160 eeeeeeeeee 3.1105313
1 161 eeeeeeeeee 3.1105313
1 162 eeeeeeeeee 3.1105313
1 163 eeeeeeeeee 3.1105313
1 164 eeeeeeeeee 3.1105313
1 165 eeeeeeeeee 3.1105313
1 166 eeeeeeeeee 3.1105313
1 167 eeeeeeeeee 3.1105313
1 168 eeeeeeeeee 3.1105313
1 169 eeeeeeeeee 3.1105313
2 0 ee         2.9655447
2 1 ee         2.9655447
2 2 ee         2.9655447
2 3 ee         2.9655447
2 4 ee         2.9655447
2 5 ee         2.9655447
2 6 ee         2.9655447
2 7 ee         2.9655447
2 8 ee         2.

4 35 tttttttttt 2.8756967
4 36 tttttttttt 2.8756967
4 37 tttttttttt 2.8756967
4 38 tttttttttt 2.8756967
4 39 tttttttttt 2.8756967
4 40 tttttttttt 2.8756967
4 41 tttttttttt 2.8756967
4 42 tttttttttt 2.8756967
4 43 tttttttttt 2.8756967
4 44 tttttttttt 2.8756967
4 45 tttttttttt 2.8756967
4 46 tttttttttt 2.8756967
4 47 tttttttttt 2.8756967
4 48 tttttttttt 2.8756967
4 49 tttttttttt 2.8756967
4 50 tttttttttt 2.8756967
4 51 tttttttttt 2.8756967
4 52 tttttttttt 2.8756967
4 53 tttttttttt 2.8756967
4 54 tttttttttt 2.8756967
4 55 tttttttttt 2.8756967
4 56 tttttttttt 2.8756967
4 57 tttttttttt 2.8756967
4 58 tttttttttt 2.8756967
4 59 tttttttttt 2.8756967
4 60 tttttttttt 2.8756967
4 61 tttttttttt 2.8756967
4 62 tttttttttt 2.8756967
4 63 tttttttttt 2.8756967
4  111 tttttttttt 2.8756967
4 112 tttttttttt 2.8756967
4 113 tttttttttt 2.8756967
4 114 tttttttttt 2.8756967
4 115 tttttttttt 2.8756967
4 116 tttttttttt 2.8756967
4 117 tttttttttt 2.8756967
4 118 tttttttttt 2.8756967
4 119 tttttttttt 2.8756967
4 

7 73            2.8370538
7 74            2.8370538
7 75            2.8370538
7 76            2.8370538
7 77            2.8370538
7 78            2.8370538
7 79            2.8370538
7 80            2.8370538
7 81            2.8370538
7 82            2.8370538
7 83            2.8370538
7 84            2.8370538
7 85            2.8370538
7 86            2.8370538
7 87            2.8370538
7 88            2.8370538
7 89            2.8370538
7 90            2.8370538
7 91            2.8370538
7 92            2.8370538
7 93            2.8370538
7 94            2.8370538
7 95            2.8370538
7 96            2.8370538
7 97            2.8370538
7 98            2.8370538
7 146            2.8370538
7 147            2.8370538
7 148            2.8370538
7 149            2.8370538
7 150            2.8370538
7 151            2.8370538
7 152            2.8370538
7 153            2.8370538
7 154            2.8370538
7 155            2.8370538
7 156            2.8370538
7 157            2.8370538


...

485 91 toem tasks 0.22923951
485 92  em tasks  0.22923951
485 93 em tasks a 0.22923951
485 94   tasks an 0.22923951
485 95  tasks and 0.22923951
485 96 tosks and  0.22923951
485 97  sks and w 0.22923951
485 98 nks and wo 0.22923951
485 99 is and wor 0.22923951
485 100 s and work 0.22923951
485 101 iind work, 0.22923951
485 102 tnd dork,  0.22923951
485 103 nd dork, b 0.22923951
485 104 d dork, bu 0.22923951
485 105  aork, but 0.22923951
485 106 tork, but  0.22923951
485 107 ork, but r 0.22923951
485 108  k, but ra 0.22923951
157 iimmensity 0.22923951
485 158 tmmensity  0.22923951
485 159 tmensity o 0.22923951
485 160  ensity of 0.22923951
485 161  nsity of  0.22923951
485 162  sity of t 0.22923951
485 163 dity of th 0.22923951
485 164 igy of the 0.22923951
485 165 ty of the  0.22923951
485 166   of the s 0.22923951
485 167 oof the se 0.22923951
485 168 tf the sea 0.22923951
485 169   the sea. 0.22923951
486 0 p you want 0.22920138
486 1 oyou want  0.22920138
486 2 tou want t 0.22920138

488 89   them tas 0.22916463
488 90 dthem task 0.22916463
488 91 toem tasks 0.22916463
488 92  em tasks  0.22916463
488 93 em tasks a 0.22916463
488 94 r tasks an 0.22916463
488 95  tasks and 0.22916463
488 96 tosks and  0.22916463
488 97  sks and w 0.22916463
488 98 nss and wo 0.22916463
488 99 ss and wor 0.22916463
488 100 , and work 0.22916463
488 101 sand work, 0.22916463
488 102 tnd work,  0.22916463
488 103 nd work, b 0.22916463
488 104 d work, bu 0.22916463
488 105  aork, but 0.22916463
488 106 tork, but  0.22916463
488 107 ork, but r 0.22916463
488 108 n , but ra 0.22916463
488 109  , but rat 0.22916463
488 110 , but rath 0.22916463
488 111  dut rathe 0.22916463
488 112 tut rather 0.22916463
488 113 ut rather  0.22916463
488 114 t rather t 0.22916463
488 115  rather te 0.22916463
488 116 tather tea 0.22916463
488 117  ther teac 0.22916463
488 118 nher teach 0.22916463
488 119  em teach  0.22916463
488 120 em teach t 0.22916463
488 121 r teach th 0.22916463
488 122  teach the 0.

491 61 tollect wo 0.22901723
491 62 hllect woo 0.22901723
491 63 nlect wood 0.22901723
491 64 eect wood  0.22901723
491 65 est wood a 0.22901723
491 66 mt wood an 0.22901723
491 67 h wood and 0.22901723
491 68  wood and  0.22901723
491 69 tord and d 0.22901723
491 70 ood and do 0.22901723
491 71 nd and don 0.22901723
491 72 n and don' 0.22901723
491 73  a d don't 0.22901723
491 74 tnd won't  0.22901723
491 75 nd won't a 0.22901723
491 76 d won't as 0.22901723
491 77  aon't ass 0.22901723
491 78 ton't assi 0.22901723
491 79  n't dssig 0.22901723
491 80 n't dssign 0.22901723
491 81 dt assign  0.22901723
491 82 t dssign t 0.22901723
491 83  wssign th 0.22901723
491 84 tnsign the em ta lon 0.22901723
491 131 er ta long 0.22901723
491 132 m ta long  0.22901723
491 133  ta long f 0.22901723
491 134 to bong fo 0.22901723
491 135   cong for 0.22901723
491 136 ncong for  0.22901723
491 137 tong for t 0.22901723
491 138 eng for th 0.22901723
491 139 n' for the 0.22901723
491 140 d for the  0.229

494 78 ton't dssi 0.22901239
494 79  n't dssig 0.22901239
494 80  't dssign 0.22901239
494 81 dt assign  0.22901239
494 82 t dssign t 0.22901239
494 83  rssign th 0.22901239
494 84 tnsign the 0.22901239
494 85 nkign them 0.22901239
494 86 iign them  0.22901239
494 87 ign them t 0.22901239
494 88 pn them ta 0.22901239
494 89   them tas 0.22901239
494 90 dthem task 0.22901239
494 91 toem tosks 0.22901239
494 92  er tasks  0.22901239
494 93 er tasks a 0.22901239
494 94 r tasks an 0.22901239
494 95  tasks and 0.22901239
494 96 tosks and  0.22901239
494 97  sks and w 0.22901239
494 98 nks and wo 0.22901239
494 99 is and wor 0.22901239
494 100 s and work 0.22901239
494 101 iind work, 0.22901239
494 102 tnd dork,  0.22901239
494 103 nd work, b 0.22901239
494 104 d work, bu 0.22901239
494 105  aork, but 0.22901239
494 106 took, but  0.22901239
494 107 ook, but r 0.22901239
494 108  k, but ra 0.22901239
494 109  , but rat 0.22901239
494 110 s but rath 0.22901239
494 111  but rathe 0.22901239
49

497 39 tp people  0.2289674
497 40 t people t 0.2289674
497 41 lpeople to 0.2289674
497 42 teople tog 0.2289674
497 43 lople toge 0.2289674
497 44 rple toget 0.2289674
497 45 nle togeth 0.2289674
497 46 le togethe 0.2289674
497 47 ectogether 0.2289674
497 48 rtogether  0.2289674
497 49 to ether t 0.2289674
497 50 h ether to 0.2289674
497 51 nether to  0.2289674
497 52 ether to c 0.2289674
497 53 rher to co 0.2289674
497 54 her to col 0.2289674
497 55 er to coll 0.2289674
497 56 r to colle 0.2289674
497 57  te collec 0.2289674
497 58 to collect 0.2289674
497 59 h collect  0.2289674
497 60 ncollect w 0.2289674
497 61 tollect wo 0.2289674
497 62 tllect woo 0.2289674
497 63 nlect wood 0.2289674
497 64 113 ut rather  0.2289674
497 114 t rather t 0.2289674
497 115 haather te 0.2289674
497 116 tather tea 0.2289674
497 117  ther teac 0.2289674
497 118 nher teach 0.2289674
497 119 her toach  0.2289674
497 120 er toach t 0.2289674
497 121 r toach th 0.2289674
497 122  teach the 0.2289674
497 123

In [16]:
chk_processting_time(time1, time2)

처리시간 : 10분 34초 경과되었습니다.


634.0412111282349

In [20]:
# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(sentence[0] + ''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')


ip you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

In [21]:
print("Original sentence : \n{}".format(sentence))

Original sentence : 
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>